In [1]:
!pip install pyomo

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory('cplex').executable = 'D:\download\glpk-4.65\w64'

In [ ]:
#SolverFactory('glpk', executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

In [16]:
#df_dmd =pd.read_excel("Desktop/717/Assignment/A3pjmdmd_bids.xlsx", 
                     #  sheet_name="hrl_dmd_bids",keep_default_na=False,header=0)

#Read generators and demand data as two dataframes  Desktop717data
df_genData=pd.read_excel("Desktop/717/data/A6Q3.xlsx",sheet_name="GeneratorsData") #read data from generators tab and skip the first row
df_demandData=pd.read_excel("Desktop/717/data/A6Q3.xlsx",sheet_name="DemandData") #read data from generators tab and skip the first row

In [4]:

df_demandData.head(5)

,Demand
0,240
1,250
2,200
3,170
4,230


In [5]:
df_genData.head(5)

,NumGen,FixedCost,SDCost,SUCost,VarCost,MaxGen,MinGen,RampDown,RampSD,RampSU,RampUp,MinUpTime,MinDownTime,ReqUp,ReqDown,InitialStatus,InitialGen
0,1,0,0,800,5,300,80,30,30,100,50,3,2,2,0,1,130
1,2,0,0,500,15,200,50,40,40,70,60,2,2,0,0,0,0
2,3,0,0,250,30,100,30,50,50,40,70,1,2,0,0,0,0


In [6]:
#define the size of the sets
NumGen=len(df_genData)
T=len(df_demandData)
print("We read technical parameters data for", NumGen, "power generating units")
print("We read demand data for", T, "time periods")

We read technical parameters data for 3 power generating units
We read demand data for 6 time periods


In [7]:

#Define Indexes
N=np.array([n for n in range(0,NumGen)]) #This array has 2 components, but the index starts at zero
T=np.array([t for t in range(0,T)])

In [8]:
T

array([0, 1, 2, 3, 4, 5])

In [9]:
df_genData.columns

Index(['NumGen', 'FixedCost', 'SDCost', 'SUCost', 'VarCost', 'MaxGen',
       'MinGen', 'RampDown', 'RampSD', 'RampSU', 'RampUp', 'MinUpTime',
       'MinDownTime', 'ReqUp', 'ReqDown', 'InitialStatus', 'InitialGen'],
      dtype='object')

In [10]:
#get the NamePlate capacity of the largest generator to use it as the upper bound of the pg,t, decision variables
#This step is optional. You can leave pg,t unbounded.
largestGen=df_genData.loc[df_genData["MaxGen"].idxmax()] #largestGen is a series object 
print('The parameters of the generator with largest Gen capacity are',largestGen)
MaxGenCapacity=largestGen.MaxGen 
print("The Name Plate Capacity of the largest generator is",MaxGenCapacity)

The parameters of the generator with largest Gen capacity are NumGen             1
FixedCost          0
SDCost             0
SUCost           800
VarCost            5
MaxGen           300
MinGen            80
RampDown          30
RampSD            30
RampSU           100
RampUp            50
MinUpTime          3
MinDownTime        2
ReqUp              2
ReqDown            0
InitialStatus      1
InitialGen       130
Name: 0, dtype: int64
The Name Plate Capacity of the largest generator is 300


In [17]:
FixCost=df_genData.loc[:,'FixedCost'].to_numpy()
SDCost=df_genData.loc[:,'SDCost'].to_numpy()
SUCost=df_genData.loc[:,'SUCost'].to_numpy()
VarCost=df_genData.loc[:,'VarCost'].to_numpy()
PMax=df_genData.loc[:,'MaxGen'].to_numpy()
PMin=df_genData.loc[:,'MinGen'].to_numpy()
RampDown=df_genData.loc[:,'RampDown'].to_numpy()
RampSD=df_genData.loc[:,'RampSD'].to_numpy()
RampSU=df_genData.loc[:,'RampSU'].to_numpy()
RampUp=df_genData.loc[:,'RampUp'].to_numpy()
InitialStatus=df_genData.loc[:,'InitialStatus'].to_numpy()
InitialGen=df_genData.loc[:,'InitialGen'].to_numpy()
Demand=df_demandData.loc[:,'Demand'].to_numpy()
#MinUpTime      
#MinDownTime    
#ReqUp 
#ReqDown  
#InitialGen 



In [18]:
print(Demand)

[240 250 200 170 230 190]


In [19]:
#UC MODEL
def Unit_Commitment():
    m=ConcreteModel()
    m.N=Set(initialize=N)
    m.T=Set(initialize=T)
    m.x=Var(m.N, m.T, bounds = (0,MaxGenCapacity))
    m.u=Var(m.N, m.T, domain=Binary)
    m.y=Var(m.N, m.T, domain=Binary)
    m.z=Var(m.N, m.T, domain=Binary)
    
    m.system_cost=Objective(expr=sum(m.x[n,t]*VarCost[n]+m.u[n,t]*FixCost[n]+ SUCost[n]*m.y[n,t] + SDCost[n]*m.z[n,t] for n in m.N for t in m.T), sense=minimize)
    
    m.DemandConstraint=Constraint(m.T, rule=lambda m, t: sum(m.x[n,t] for n in N) == Demand[t])
    
    m.MinPower=Constraint(m.N, m.T, rule=lambda m, n, t: PMin[n]*m.u[n,t] <= m.x[n,t])
    m.MaxPower=Constraint(m.N, m.T, rule=lambda m, n, t: PMax[n]*m.u[n,t] >= m.x[n,t])
    
    # Constraint to ensure y and z are mutually exclusive for each unit and time period ok
    m.StartShutExclusive = Constraint(m.N, m.T, rule=lambda m, n, t: m.y[n,t] + m.z[n,t] <= 1)
    
    # Constraint to ensure that for each unit and time period, y minus z equals u at t minus u at t-1
    
    m.StartStop = Constraint(m.N, m.T, rule=lambda m, n, t:
                             m.y[n,t] - m.z[n,t] == m.u[n,t] - m.u[n,t-1] if t > 0 else
                             m.y[n,t] - m.z[n,t] == m.u[n,t] - InitialStatus[n])


    # Add constraints on ramping-up feasibility , when t=0 and t>0
    m.RampUpConstraint = Constraint(m.N, m.T, rule=lambda m, n, t:
                                     m.x[n,t] - m.x[n,t-1] <= RampUp[n]*m.u[n,t] + RampSU[n]*m.y[n,t] if t>0 else
                                     m.x[n,t] - InitialGen[n] <= RampUp[n]*InitialStatus[n] + RampSU[n]*m.y[n,t])

    
    # Add constraints on ramping-up feasibility , when t=0 and t>0
    m.RampDownConstraint = Constraint(m.N, m.T, rule=lambda m, n, t:
                                     m.x[n,t-1] - m.x[n,t] <= RampDown[n]*m.u[n,t] + RampSD[n]*m.z[n,t] if t>0 else 
                                     InitialGen[n] - m.x[n,t] <= RampDown[n]*m.u[n,t] + RampSD[n]*m.z[n,t])
    


    return m




In [21]:
#m=Unit_Commitment()
#SolverFactory('glpk').solve(m).write() # Instead of this line, write "opt.solve(m)" if model's details are not important

#m = Unit_Commitment()
#SolverFactory('glpk').opt.solve(m)

m=Unit_Commitment()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 10300.0
  Upper bound: 10300.0
  Number of objectives: 1
  Number of constraints: 115
  Number of variables: 73
  Number of nonzeros: 331
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 15
      Number of created subproblems: 15
  Error rc: 0
  Time: 0.09446525573730469
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [22]:
print('SOLUTION')
print('The total system cost is = $',m.system_cost())
print('Commitment and Generation')
for n in N:
    for t in T:
        print("n = {0:2d}  t = {1:2d}   {2} {3:.0f}   {4:.0f}   {5:.0f}  {6:.2f}".format(n, t, m.u[n,t], m.u[n,t](),m.y[n,t](),m.z[n,t](), m.x[n,t]()))
        #Line above makes columns the same width and formats the numbers of decimals for u and x. Line below has a messy format
        #print(m.x[n,t]," = ", m.x[n,t](), "MWh", m.u[n,t], " =", m.u[n,t]()) #this works too but format is messy

SOLUTION
The total system cost is = $ 10300.0
Commitment and Generation
n =  0  t =  0   u[0,0] 1   0   0  180.00
n =  0  t =  1   u[0,1] 1   0   0  180.00
n =  0  t =  2   u[0,2] 1   0   0  150.00
n =  0  t =  3   u[0,3] 1   0   0  120.00
n =  0  t =  4   u[0,4] 1   0   0  170.00
n =  0  t =  5   u[0,5] 1   0   0  140.00
n =  1  t =  0   u[1,0] 1   1   0  60.00
n =  1  t =  1   u[1,1] 1   0   0  70.00
n =  1  t =  2   u[1,2] 1   0   0  50.00
n =  1  t =  3   u[1,3] 1   0   0  50.00
n =  1  t =  4   u[1,4] 1   0   0  60.00
n =  1  t =  5   u[1,5] 1   0   0  50.00
n =  2  t =  0   u[2,0] 0   0   0  0.00
n =  2  t =  1   u[2,1] 0   0   0  0.00
n =  2  t =  2   u[2,2] 0   0   0  0.00
n =  2  t =  3   u[2,3] 0   0   0  0.00
n =  2  t =  4   u[2,4] 0   0   0  0.00
n =  2  t =  5   u[2,5] 0   0   0  0.00
